<h1 style='color:Green'>Feature Engineering – Time-Based Signals</h1>
<h3>📌 Objectives </h3>
<pre>
    Transform cleaned raw data into high-signal features that capture:
    - Temporal patterns
    - Promotion effects
    - Holiday impact
    - Store & product behavior
    - Lagged demand dynamics
</pre>

<h2 style='color:Green'>Time Based Feature Engineering</h2>
<h3>📌 Objectives </h3>
<pre>
   Capture calendar-driven demand patterns:
    - Weekly cycles
    - Monthly seasonality
    - Weekend behavior
    - Salary payment effects (domain knowledge)
</pre>

<h2 style='color:purple'>Import Essentials</h2>

<h3 style='color:purple'>Core Time-Based Features (Day, Week, Month, Year)</h3>
<pre>
    These are foundational features.
    - Sales differ by weekday
    - Monthly seasonality is strong in retail
    - Long-term trends captured by year
</pre>

In [2]:
# Import and Load Data 
import pandas as pd 
import numpy as np 
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path

In [3]:
# project path
PROJECT_ROOT = Path.cwd().parent
FEATURE_DATA = PROJECT_ROOT / 'data' / "features"

In [ ]:
# Load data 
train = pd.read_parquet(FEATURE_DATA / "train_features.parquet")